```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Static Layer](#static-layer)**


# **FLOW DIAGRAMS**

---


Required steps:



- **<font color=teal>SL</font>**: Static Layer


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Schemas.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]'))
CREATE TABLE [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] (
	[ID] [BIGINT] IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[CAMPAIGN_LABEL_NL] [VARCHAR](255),
	[CAMPAIGN_LABEL_EN] [VARCHAR](255),
	[CAMPAIGN_CODE] [VARCHAR](255),
	[ORDER] [VARCHAR](100)
);
GO

# **STATIC LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]'))
CREATE TABLE [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[CAMPAIGN_LABEL_NL] [VARCHAR](255),
	[CAMPAIGN_LABEL_EN] [VARCHAR](255),
	[CAMPAIGN_CODE] [VARCHAR](255),
	[ORDER] INT
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]
	ON [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]
GO

CREATE INDEX [NCIX_DLI_RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]
	ON [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] (
		[DATE_LAST_INSERTED]
	);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] (
        [CAMPAIGN_LABEL_NL],
        [CAMPAIGN_LABEL_EN],
        [CAMPAIGN_CODE],
        [ORDER]
    )
    SELECT
        [CAMPAIGN_LABEL_NL],
        [CAMPAIGN_LABEL_EN],
        [CAMPAIGN_CODE],
        [ORDER]
    FROM
        [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA])
END;
GO